# Car Price Prediction::

Download dataset from this link:

https://www.kaggle.com/hellbuoy/car-price-prediction

# Problem Statement::

A Chinese automobile company Geely Auto aspires to enter the US market by setting up their manufacturing unit there and producing cars locally to give competition to their US and European counterparts.

They have contracted an automobile consulting company to understand the factors on which the pricing of cars depends. Specifically, they want to understand the factors affecting the pricing of cars in the American market, since those may be very different from the Chinese market. The company wants to know:

Which variables are significant in predicting the price of a car
How well those variables describe the price of a car
Based on various market surveys, the consulting firm has gathered a large data set of different types of cars across the America market.

# task::
We are required to model the price of cars with the available independent variables. It will be used by the management to understand how exactly the prices vary with the independent variables. They can accordingly manipulate the design of the cars, the business strategy etc. to meet certain price levels. Further, the model will be a good way for management to understand the pricing dynamics of a new market.

# WORKFLOW ::

1.Load Data          ################ DONE

2.Check Missing Values ( If Exist ; Fill each record with mean of its feature ) ############# DONE

3.Split into 50% Training(Samples,Labels) , 30% Test(Samples,Labels) and 20% Validation Data(Samples,Labels).

4.Model : input Layer (No. of features ), 3 hidden layers including 10,8,6 unit & Output Layer with activation function relu/tanh (check by experiment).

5.Compilation Step (Note : Its a Regression problem , select loss , metrics according to it)
6.Train the Model with Epochs (100) and validate it

7.If the model gets overfit tune your model by changing the units , No. of layers , activation function , epochs , add dropout layer or add Regularizer according to the need .

8.Evaluation Step

9.Prediction

In [1]:
import numpy as np
import pandas as pd

In [20]:
df = pd.read_csv(r"C:\Users\hc\Documents\GitHub\AI-Q2-learning-resources\DLAssignments\data\Car Price Prediction\CarPrice_Assignment.csv")

In [3]:
df.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [4]:
df.shape # 205 records, 26 features

(205, 26)

In [5]:
# checking missing values
df.isnull().sum() # 0

car_ID              0
symboling           0
CarName             0
fueltype            0
aspiration          0
doornumber          0
carbody             0
drivewheel          0
enginelocation      0
wheelbase           0
carlength           0
carwidth            0
carheight           0
curbweight          0
enginetype          0
cylindernumber      0
enginesize          0
fuelsystem          0
boreratio           0
stroke              0
compressionratio    0
horsepower          0
peakrpm             0
citympg             0
highwaympg          0
price               0
dtype: int64

In [6]:
# cross checking
df.info() # no null value

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   car_ID            205 non-null    int64  
 1   symboling         205 non-null    int64  
 2   CarName           205 non-null    object 
 3   fueltype          205 non-null    object 
 4   aspiration        205 non-null    object 
 5   doornumber        205 non-null    object 
 6   carbody           205 non-null    object 
 7   drivewheel        205 non-null    object 
 8   enginelocation    205 non-null    object 
 9   wheelbase         205 non-null    float64
 10  carlength         205 non-null    float64
 11  carwidth          205 non-null    float64
 12  carheight         205 non-null    float64
 13  curbweight        205 non-null    int64  
 14  enginetype        205 non-null    object 
 15  cylindernumber    205 non-null    object 
 16  enginesize        205 non-null    int64  
 1

In [24]:
data = df.loc[:, 'symboling':'highwaympg'] # not including Car_ID because it plays no role in predicting price
targets = df.loc[:, 'price']

In [23]:
# shuffling data
# https://stackoverflow.com/questions/29576430/shuffle-dataframe-rows
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.reset_index.html
data = data.sample(frac=1).reset_index(drop=True) #  drop = True will prevent adding an index column
data

,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,...,cylindernumber,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg
0,2,volkswagen 411 (sw),gas,std,four,sedan,fwd,front,97.3,171.7,...,four,109,mpfi,3.19,3.40,9.0,85,5250,27,34
1,-1,toyouta tercel,gas,std,four,wagon,rwd,front,104.5,187.8,...,six,161,mpfi,3.27,3.35,9.2,156,5200,19,24
2,-1,toyota mark ii,gas,std,four,hatchback,fwd,front,102.4,175.6,...,four,122,mpfi,3.31,3.54,8.7,92,4200,27,32
3,0,jaguar xj,gas,std,four,sedan,rwd,front,113.0,199.6,...,six,258,mpfi,3.63,4.17,8.1,176,4750,15,19
4,-1,volvo 145e (sw),gas,std,four,sedan,rwd,front,109.1,188.8,...,four,141,mpfi,3.78,3.15,9.5,114,5400,23,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,3,porsche macan,gas,std,two,hatchback,rwd,front,94.5,168.9,...,four,151,mpfi,3.94,3.11,9.5,143,5500,19,27
201,2,volkswagen model 111,diesel,std,four,sedan,fwd,front,97.3,171.7,...,four,97,idi,3.01,3.40,23.0,52,4800,37,46
202,3,mitsubishi g4,gas,std,two,hatchback,fwd,front,96.3,173.0,...,four,122,2bbl,3.35,3.46,8.5,88,5000,25,32
203,2,saab 99le,gas,std,four,sedan,fwd,front,99.1,186.6,...,four,121,mpfi,3.54,3.07,9.3,110,5250,21,28


In [27]:
# Split into 50% Training(Samples,Labels) , 30% Test(Samples,Labels) and 20% Validation Data(Samples,Labels).

# 50 + 20 => 70 percent for training and validation
# 70 percent of 205 = 143.5

train_data = data.iloc[:143] ############# 0 based so end at 143 @!%$#&^$&!@$
train_targets = targets.iloc[:143]


# 30 percent for test
# 205 - 143 = 62
test_data = data.iloc[143:]
test_targets = targets.iloc[143:]

# cross checking
print(train_data.shape)
print(train_targets.shape)

print(test_data.shape)
print(test_targets.shape)


(143, 24)
(143,)
(62, 24)
(62,)


# Exploring Data

In [ ]:
data.head()

In [ ]:
target.head()

## Correlation - Feature Selection

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.figure(figsize=(12, 10))

sns.heatmap(data.corr(), annot=True)

In [ ]:
## manually cross checking correlated columns

# threshold = 85%

# these are the columns having correlation > threshold
# symboling -- wheelbase # NOOOOOOOOOOOOOOO
# car length -- wheelbase
# curbweight -- car length
# city mpg --- highway mpg
# car length --- car width
# curbweight -- car width




# wheelbase --- carlength --- city mpg ---- highway mpg --- curbweight --- car width --- engine size

In [ ]:
# with the following function we can select highly correlated features
# it will remove the first feature that is correlated with anything other feature

def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            ### not including abs because it will also take into account negatively correlated features
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value 
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)

    return col_corr

In [ ]:
correlation(data, 0.85) # 85 percent is threshold

In [ ]:
data.corr() > 0.85

In [ ]:
x = data.drop({'carlength', 'curbweight', 'highwaympg'}, 1)

In [ ]:
x.corr()>0.85

In [ ]:
# Split into 50% Training(Samples,Labels) , 30% Test(Samples,Labels) and 20% Validation Data(Samples,Labels)

## shuffling the data before
